# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [ ]:
# TODO: Install any packages that you might need

In [2]:
# TODO: Import any packages that you might need
from tqdm import tqdm


## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [8]:
import os
import json
import boto3

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

  0%|          | 0/1228 [00:00<?, ?it/s]

  0%|          | 1/2299 [00:00<05:52,  6.52it/s]

  0%|          | 1/2666 [00:00<05:38,  7.87it/s]

  0%|          | 2/2373 [00:00<03:41, 10.71it/s]

  0%|          | 2/1875 [00:00<02:13, 14.08it/s]

100%|██████████| 1875/1875 [03:05<00:00, 10.12it/s]


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [3]:
#TODO: Perform any data cleaning or data preprocessing
from os import path, listdir, makedirs
from random import sample
import glob
from tqdm import tqdm

def directory_to_dictionay(directory):
    final_dict = {}
    
    
    
def train_test_split(dataset_path, per_test, per_val):
    classes_directory = listdir(dataset_path)
    result_classes_split = {'train':{ class_dir: [] for class_dir in classes_directory},
                                            'val':{ class_dir: [] for class_dir in classes_directory},
                                           'test':{ class_dir: [] for class_dir in classes_directory}}
    if per_test + per_val >= 100:
        raise Exception( 'ERROR')
    
    for class_directory in classes_directory:
        elements_class =  glob.glob(f'{path.join(dataset_path, class_directory)}/*')
        n_elements_class = len(elements_class)
        n_sample_test  =  (n_elements_class * per_test) //100
        n_sample_val  =  (n_elements_class * per_val) //100
        result_classes_split['test'][class_directory] = elements_class[:n_sample_test -1 ]
        result_classes_split['val'][class_directory] = elements_class[n_sample_test - 1 : n_sample_test  + n_sample_val - 2]
        result_classes_split['train'][class_directory] = elements_class[n_sample_test  + n_sample_val - 2 : ]
        # length = len(result_classes_split['test'][class_directory] ) + len(result_classes_split['val'][class_directory])  + len(result_classes_split['train'][class_directory])
        # assert length == len(elements_class)  ,'convertion error'
        
    return result_classes_split

def write_sample_elements(dir_name, data):
    makedirs(dir_name, exist_ok= True)
    classes = list(data.keys())
    for class_name in classes:
        dir_class = path.join(dir_name, class_name)
        makedirs(dir_class, exist_ok=True)
        for file_path in tqdm(data[class_name]):
            filename_output = path.join(path.join(dir_name, class_name), file_path.split('/')[-1])
            with open(file_path, 'rb') as img_input, open(filename_output, 'wb') as img_output:
                img_output.write(img_input.read())

In [5]:
splitted_data = train_test_split('train_data/', 30, 10)
base_directory = 'BinImages'
for dataset_name  in splitted_data:
    dir_name = path.join(base_directory,  dataset_name)    
    write_sample_elements(dir_name,  splitted_data[dataset_name])

 44%|████▍     | 615/1383 [00:29<00:13, 55.30it/s]

In [27]:
test.keys()

dict_keys(['4', '2', '1', '5', '3'])

In [47]:

#!aws s3 cp  train_data  s3://lucialedezmacapstoneproject/BinImages/  --recursive

# import sagemaker
# bucket = "lucialedezmacapstone"
# sagemaker_session = sagemaker.Session()
# inputs = sagemaker_session.upload_data(path="train_data", bucket=bucket, key_prefix="train_data_2")

100%|██████████| 265/265 [00:03<00:00, 68.85it/s]


## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [ ]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values
hyperparameters = {'batch_size':  64,  'learning_rate':  0.01}

In [ ]:
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point='train.py',
    base_job_name='bin_estimator',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters)

In [ ]:
# TODO: Fit your estimator

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances